In [1]:
from multiprocess import Pool
from typing import List, Dict, Tuple, Callable, Iterable

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

In [2]:
import re
import nltk
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Uncomment if you are using colab

In [3]:
# !mkdir ./data
# !wget https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture08/data/train.csv.zip -O ./data/train.csv.zip
# !wget https://raw.githubusercontent.com/vadim0912/MLIntro2021/main/lecture08/data/test.csv.zip -O ./data/test.csv.zip

Я просто попытался в лоб решить, посчитал вероятности встречаемости для 2х и 3х слов идущих вместе, потом проходил по тестовому датафрейму, и подбирал самые вероятные слова с подходящим префиксом, сначала в модели для 2х слов, а если таких не было, то подбирал по последнему слову и префиксу, а если и там не находил, то просто ставил самое часто встречающаееся слово с таким префиксом. Никакой регрессии, никакого градиентного спуска, никакой кластеризации по языкам, наверное поэтому так долго считается...

In [4]:
df = pd.read_csv("./data/train.csv.zip")

In [5]:
df.head()

,sentence,language
0,После увольнения я решил собрать своих друзей ...,ru
1,"Ты знаешь , что это не гнев заставил моё сердц...",ru
2,Просто има толкова много други хора в пъпа ти ...,bg
3,"Ради всех имён , которые я никогда не узнаю , ...",ru
4,Ми пропускаємо найважливіші для нас речі . Най...,uk


In [6]:
class TrieNode:
    
    def __init__(self, char: str) -> None:
        self.char: str = char
        self.is_end: bool = False
        self.count: int = 0
        self.children: Dict = {}


class Trie:

    def __init__(self, words: Iterable[str] = None):
        self.root = TrieNode("")
        
        if words:
            for word in words:
                self.insert(word)
        
    def insert(self, word: str) -> None:
        
        node: TrieNode = self.root
        
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                new_node: TrieNode = TrieNode(char)
                node.children[char] = new_node
                node = new_node
        node.is_end = True
        node.count += 1
        
    def dfs(self, node, prefix):
        
        if node.is_end:
            self.output.append((prefix + node.char, node.count))
            
        for child in node.children.values():
            self.dfs(child, prefix + node.char)
    
    def query(self, prefix: str) -> List[Tuple[str, int]]:
        
        self.output = []
        node = self.root
        
        for char in prefix:
            if char in node.children:
                node = node.children[char]
            else:
                return []
            
        self.dfs(node, prefix[:-1])
        
        return sorted(self.output, key=lambda x: -x[1])


In [7]:
trie = Trie(word for sentence in df['sentence'].values for word in sentence.lower().split())

In [8]:
trie.query('шовкогра')

[]

In [9]:
test_df = pd.read_csv("./data/test.csv.zip")

In [ ]:
def normalize_and_tokenize(text: str) -> List[str]:
    text = text.lower().replace('ё', 'е')
    text = re.sub(' +', ' ', text).strip()
    text = nltk.wordpunct_tokenize(text)
    return text

In [ ]:
tokenized_texts = df["sentence"].apply(normalize_and_tokenize).values.tolist()

In [ ]:
BOS = '<BOS>'
EOS = '<EOS>'

ngrams_config = {
    "pad_left": True,
    "pad_right": True,
    "left_pad_symbol": BOS,
    "right_pad_symbol": EOS
}

def build_ngram_counts(tokenized_texts: Iterable[Iterable[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, int]]:    
    counts = defaultdict(Counter)
    for tokens in tokenized_texts:
        for ngram in nltk.ngrams(tokens, n=n, **ngrams_config):
            prefix, next_token = ngram[:-1], ngram[-1]
            counts[prefix][next_token] += 1
    
    return counts

In [ ]:
class LanguageModel:
    
    def __init__(self, tokenized_texts: Iterable[Iterable[str]], n: int) -> None:
        
        self.n: int = n
        self.probs: Dict[Tuple[str, ...], Dict[str, float]] = defaultdict(Counter)
        
        for prefix, distribution in build_ngram_counts(tokenized_texts, n=n).items():
            norm = sum(distribution.values())
            self.probs[prefix] = Counter({
                token: count / norm for token, count in distribution.items()
            })
            
    def get_token_distribution(self, prefix: List[str]) -> Dict[str, float]:
        
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [BOS] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix: List[str], token: str) -> float:
        return self.get_token_distribution(prefix)[token]

In [ ]:
model = LanguageModel(tokenized_texts, n=3)

In [ ]:
model_one = LanguageModel(tokenized_texts, n =2)

In [ ]:
def retrieve_last_words(sentence: str) -> List[str]:
    sentence = sentence.lower().replace('ё', 'е')
    sentence = re.sub(' +', ' ', sentence).strip()
    sentence = nltk.wordpunct_tokenize(sentence)
    
    return sentence[:-4:-1]

last_words = test_df["prefix"].apply(retrieve_last_words)

tokens = []
for words in last_words:
    prefix = words[0]
    words = [BOS] * (3 - len(words)) + words
    data = sorted(model.probs[(words[2], words[1])].items(), key=lambda x: -x[1])
    i = 0
    size_data = len(data)
    while i < size_data and not data[i][0].startswith(prefix):
        i += 1
    if (i == size_data):
        j = 0
        data = sorted(model_one.probs[(words[1],)].items(), key=lambda x: -x[1])
        size_data = len(data)
        while j < size_data and not data[j][0].startswith(prefix):
            j += 1
        if j == size_data:
            if trie.query(prefix):
                tokens.append(trie.query(prefix)[0][0])
            else:
                tokens.append(prefix)
        else:
            tokens.append(data[j][0])

    else:
        tokens.append(data[i][0])

In [ ]:
t = 173
print(test_df["prefix"][t])
print(last_words[t])
print(tokens[t])

Я можу виготовляти на замовлення з допомогою усіх видів модних технологій , в &apos; язання , лазерна нарізка , шовкогра
['шовкогра', ',', 'нарізка']
шовкогра


In [ ]:
len(tokens)

266618

In [ ]:
data_for_df = {'index': range(len(tokens)), 'token': tokens}
test_prediction_df = pd.DataFrame(data_for_df)

In [ ]:
test_prediction_df

,index,token
0,0,процента
1,1,спортзал
2,2,осіб
3,3,токсичных
4,4,имя
...,...,...
266613,266613,края
266614,266614,негово
266615,266615,общей
266616,266616,пътя


In [ ]:
test_prediction_df[['index', 'token']].to_csv("simple_baseline.csv", index=False)